In [1]:
import pandas as pd
import numpy as np
import matplotlib
import math
matplotlib.use("Agg")
from matplotlib import pyplot as plt

In [2]:
np.random.seed(42)

class Scaler():
    # hint: https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/
    def _init_(self):
        raise NotImplementedError
    def _call_(self,features, is_train=False):
        if is_train==True:
            self.mean = np.mean(features, axis=0)
            self.max= np.max(features,axis=0) 
            self.min= np.min(features,axis=0)
            features = (features - self.min)/(self.max-self.min) 
        else:
            features =(features - self.min)/(self.max-self.min)
            
        return features
        raise NotImplementedError

In [3]:
def get_features(csv_path,is_train=False,scaler=None):
    '''
    Description:
    read input feature columns from csv file
    manipulate feature columns, create basis functions, do feature scaling etc.
    return a feature matrix (numpy array) of shape m x n 
    m is number of examples, n is number of features
    return value: numpy array
    '''

    '''
    Arguments:
    csv_path: path to csv file
    is_train: True if using training data (optional)
    scaler: a class object for doing feature scaling (optional)
    '''

    '''
    help:
    useful links: 
        * https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
        * https://www.geeksforgeeks.org/python-read-csv-using-pandas-read_csv/
    '''
    #path = str(csv_path)
    
    #df_data = pd.read_csv(filepath_or_buffer = "%s" % csv_path)
  
    df_data = pd.read_csv(filepath_or_buffer = csv_path)
    
    #pre- processing data
    df_data = df_data.iloc[: , 1:]
    df_data = df_data.dropna()
    
    #Feature Engineering
    df_old = df_data
    df_data["daynight"] = df_data["daynight"].apply(lambda val: 1 if val == "D" else 0)
    df_data["satellite"] = df_data["satellite"].apply(lambda val: 1 if val == "Terra" else 0)
    data = df_data.copy()
    #Numerical Features for Date 
    df_data["acq_date"] = pd.to_datetime(df_data["acq_date"])
    df_data["acq_day"] = df_data["acq_date"].apply(lambda row: row.day)
    df_data["acq_date"] = df_data["acq_date"].apply(lambda row: row.month)
    
    #df_data["acq_year"] = df_data["acq_date"].apply(lambda row: row.year)
    
    df_data = df_data.drop(["instrument","version"],axis=1)
    df_data["confidence"] = df_data["confidence"].astype("float")
    df_data["latitude"] = df_data["latitude"]*(-1)
    df_data["acq_time"] = df_data["acq_time"].astype("float")
    
    #Drop co-related columns
    #df_data["track"] = df_data["track"] - df_data["scan"]
    #df_data["track"] = df_data["track"]**2
    #df_data["scan"] = df_data["scan"]**2
    #df_data["confidence"] = df_data["confidence"]**2
    #df_data["bright_t31"] = df_data["bright_t31"]**2
   
    #df_data["bright"] = 
    #log normalization
    #df_data["brightness"]=np.log(df_data["brightness"])
    #df_data["bright_t31"]=np.log(df_data["bright_t31"])
    #Normalise data
    #df_data_norm = df_data/((df_data.max()-df_data.min())
    #df_data_norm = df_data
    #df_data["base"] = 1
    #scan and track are corelated
    
    #Normalize

    #for i in range(len(df_data.columns)):
    #    df_data.iloc[: , i] = (df_data.iloc[: , i] - df_data.iloc[: , i].mean())/df_data.iloc[: , i].std()

    df_data["acq_day_period"] = df_data["acq_day"]
    #df_data["acq_date_period"] = df_data["acq_date"]
    df_data["track_period"] = df_data["track"]
    #extract periodicity
    for i in range(df_data.shape[0]):
        df_data.loc[i, "acq_day_period"] = math.sin(df_data.loc[i, "acq_day"])
        #df_data.loc[i, "acq_date_period"] = math.sin(df_data.loc[i, "acq_date"])
        df_data.loc[i, "track_period"] = math.sin(df_data.loc[i, "track"])
    #Normalize
    df_data = (df_data-df_data.min())/(df_data.max()-df_data.min())
    
    df_data = df_data.drop(["track"],axis=1)
        
    df_data["daynight"] = data ["daynight"]
    df_data["satellite"] = data["satellite"]
    df_data["base"] = 1

    first_column = df_data.pop('base')

    df_data.insert(0, 'base', first_column)
    basis_1 = True
    basis_2 = False

    if basis_1 == True:
      df_data = df_data**2
    
    if basis_2 == True:
      df_data = df_data**3
    
    if is_train == True:
      df_data = df_data.drop(["frp"],axis=1)
        
    df_data = df_data.to_numpy()
    return(df_data)

    raise NotImplementedError

In [4]:
feature_matrix = get_features("train.csv",True)

#df_features = df_features.Scaler()

# feature_matrix.head()

# feature_matrix.var()

In [5]:
def get_targets(csv_path):
    '''
    Description:
    read target outputs from the csv file
    return a numpy array of shape m x 1
    m is number of examples
    '''
    df_target = pd.read_csv(filepath_or_buffer = csv_path)
    
    #pre- processing data
    df_target = df_target.dropna()
    df_target = df_target["frp"]

    df_target = df_target.to_numpy()
    return(df_target)
    raise NotImplementedError

In [6]:
targets = get_targets("train.csv")



In [7]:
def analytical_solution(feature_matrix, targets, C=0.0):
    '''
    Description:
    implement analytical solution to obtain weights
    as described in lecture 5d
    return value: numpy array
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    targets: numpy array of shape m x 1
    '''
    # Get transpose
    feature_transpose = feature_matrix.T
    
    n = feature_matrix.shape[1]
    # Multiply the matrices: Matrix1 and Matrix2
    result = np.matmul(feature_transpose,feature_matrix);
    
    result = result + (np.identity(n))*C
    #invert
    result_inv = np.linalg.inv(result)
    print(n)
    output = np.matmul(result_inv,feature_transpose)
    
    weights = np.matmul(output,targets)
    
    
    return weights

    raise NotImplementedError 

In [8]:

weights = analytical_solution(feature_matrix, targets, 0)
weights

14


array([ -47.84793995,    5.14092064,   -5.4048751 , 1887.08509411,
        244.03920169,   11.02879454,   14.16281237,   -1.92149527,
          2.63845817, -127.94397905,  -18.81447254,   -7.7145926 ,
         25.09225444,   91.78053648])

In [9]:

def get_predictions(feature_matrix, weights):
    '''
    description
    return predictions given feature matrix and weights
    return value: numpy array
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    '''
    #test_features = get_features(feature_matrix)
    
    predictions = np.matmul(feature_matrix,weights)
    return (predictions)
    raise NotImplementedError




In [10]:

dev_features = get_features("dev.csv",True)
dev_features.shape

dev_targets = get_targets("dev.csv")

weights =  analytical_solution(dev_features, dev_targets, 0)

14


In [11]:
predictions = get_predictions(dev_features, weights)
predictions


array([159.51642724,  40.6739243 ,  48.7448923 , ..., -34.22917484,
       113.49263834,  -9.37229991])

In [12]:
feature_matrix = get_features("train.csv",True)
targets = get_targets('train.csv')
weights =   analytical_solution(feature_matrix, targets, 0)

14


In [13]:
def mse_loss(feature_matrix, weights, targets):
    '''
    Description:
    Implement mean squared error loss function
    return value: float (scalar)
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    targets: numpy array of shape m x 1
    '''

    predictions = np.matmul(feature_matrix,weights)
    error = targets - predictions
    squared_error = np.sum(np.square(error))
    
    mse = squared_error/len(targets)

    return mse
    raise NotImplementedError

In [14]:

mse_loss(dev_features, weights, dev_targets)

35088.52860675592

In [15]:
feature_matrix = get_features("dev.csv",True)
targets = get_targets('dev.csv')
weights =   analytical_solution(feature_matrix, targets, 0)

14


In [16]:

def l2_regularizer(weights):
    '''
    Description:
    Implement l2 regularizer
    return value: float (scalar)
    '''

    '''
    Arguments
    weights: numpy array of shape n x 1
    '''
    
    #df_reg = weights
    #df_reg_sq = weights**2
    l2_regular =np.sum(np.square(weights[:-1]))
    #return(df_reg_sq.sum())
    return l2_regular
    raise NotImplementedError



In [17]:

print(l2_regularizer(weights))

4195613.494906143


In [18]:

def loss_fn(feature_matrix, weights, targets, C=0.0):
    '''
    Description:
    compute the loss function: mse_loss + C * l2_regularizer
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    targets: numpy array of shape m x 1
    C: weight for regularization penalty
    return value: float (scalar)
    '''
    loss = mse_loss(feature_matrix, weights, targets)
    reg = l2_regularizer(weights)
    c_reg = C*reg
    return(loss+c_reg)
    raise NotImplementedError

In [19]:
loss_fn(feature_matrix, weights, targets,1)

4230240.707823914

In [20]:

def sample_random_batch(feature_matrix, targets, batch_size):
    '''
    Description
    Batching -- Randomly sample batch_size number of elements from feature_matrix and targets
    return a tuple: (sampled_feature_matrix, sampled_targets)
    sampled_feature_matrix: numpy array of shape batch_size x n
    sampled_targets: numpy array of shape batch_size x 1
    '''

    feature_random = np.random.choice(feature_matrix.shape[0], size=batch_size, replace=False)
    
    # sampled_feature_matrix = feature_matrix.loc[feature_random,:]
    # sampled_targets=targets[feature_random]
    '''
    
    Arguments:
    feature_matrix: numpy array of shape m x n
    targets: numpy array of shape m x 1
    batch_size: int
    '''   


    return (feature_matrix[feature_random],targets[feature_random])
    # return (sampled_feature_matrix,sampled_targets)
    raise NotImplementedError

In [21]:

sample_random_batch(feature_matrix, targets, batch_size=100)

(array([[1.00000000e+00, 2.35572155e-02, 5.54438181e-01, ...,
         5.87777778e-01, 2.24950017e-03, 3.28149994e-01],
        [1.00000000e+00, 2.36360186e-02, 2.83248353e-01, ...,
         2.50000000e-01, 1.27967311e-01, 9.02455996e-01],
        [1.00000000e+00, 3.86042049e-02, 3.66626749e-01, ...,
         5.87777778e-01, 2.24950017e-03, 3.28149994e-01],
        ...,
        [1.00000000e+00, 6.13692321e-01, 8.38990108e-01, ...,
         1.77777778e-02, 4.25594914e-04, 9.89623815e-02],
        [1.00000000e+00, 5.03090221e-01, 8.76594292e-01, ...,
         1.77777778e-02, 4.25594914e-04, 0.00000000e+00],
        [1.00000000e+00, 6.17667279e-01, 7.38241519e-01, ...,
         1.11111111e-03, 9.19974487e-01, 1.00000000e+00]]),
 array([2.50000e+01, 4.56000e+01, 5.78000e+01, 1.08000e+01, 9.44000e+01,
        1.84000e+01, 8.47300e+02, 1.45000e+01, 3.13100e+02, 5.03000e+01,
        3.99000e+01, 6.09000e+01, 8.50000e+00, 2.20000e+01, 1.05700e+02,
        4.75000e+01, 2.09000e+01, 7.43000e+01,

In [22]:
def initialize_weights(n):
    '''
    Description:
    initialize weights to some initial values
    return value: numpy array of shape n x 1
    '''

    '''
    Arguments
    n: int
    '''
    return(np.zeros((n, 1)))
    
    raise NotImplementedError



In [23]:

def early_stopping(arg_1=None, arg_2=None, arg_3=None, arg_n=None):
    # allowed to modify argument list as per your need
    # return True or False
    raise NotImplementedError

In [24]:

    

def plot_trainsize_losses():
    '''
    Description:
    plot losses on the development set instances as a function of training set size 
    '''

    '''
    Arguments:
    # you are allowed to change the argument list any way you like 
    '''    

    raise NotImplementedError



In [25]:

def update_weights(weights, gradients, lr):
    '''
    Description:
    update weights using gradient descent
    retuen value: numpy matrix of shape nx1
    '''

    '''
    Arguments:
    # weights: numpy matrix of shape nx1
    # gradients: numpy matrix of shape nx1
    # lr: learning rate
    '''    
    
    #Minimize the function
    updated_weights = weights - gradients*lr
    return(updated_weights)
    raise NotImplementedError



In [26]:
feature_matrix[:,0]

array([1., 1., 1., ..., 1., 1., 1.])

In [27]:


def compute_gradients(feature_matrix, weights, targets, C=0.0):
    '''
    Description:
    compute gradient of weights w.r.t. the loss_fn function implemented above
    '''

    '''
    Arguments:
    feature_matrix: numpy array of shape m x n
    weights: numpy array of shape n x 1
    targets: numpy array of shape m x 1
    C: weight for regularization penalty
    return value: numpy array
    '''
    
    predictions = np.matmul(feature_matrix,weights)
    
    error = (targets - predictions)
    
    gradient = weights*0
    
    feature_i = feature_matrix[:,0]
    
    for i in range(len(weights)):
            
        feature_i = feature_matrix[:,i]
        gradient[i] = (np.matmul(feature_i.reshape(1,len(feature_i)),error))*(-2)
        #Dont penalize intercept
        if i == 0:
            gradient[i] = gradient[i] 
        else:
            gradient[i] = gradient[i] + (2*C*weights[i])
    return (gradient/len(feature_i))
    ##Implement ridge regression
    
    raise NotImplementedError



In [28]:
compute_gradients(feature_matrix, weights, targets, C=0.0)

array([ 2.72780215e-15, -1.42073029e-14,  6.09209148e-14,  5.60336026e-14,
       -9.31999070e-15, -9.31999070e-15,  1.00161485e-13,  6.97862718e-14,
        6.67174944e-14,  3.08014327e-14,  6.43306675e-14, -1.47755950e-15,
        2.18224172e-14, -1.81853477e-14])

In [29]:
train_features = get_features("train.csv",True)

train_targets = get_targets("train.csv")

dev_features = get_features("dev.csv",True)

dev_targets = get_targets("dev.csv")


In [32]:
def do_gradient_descent(train_feature_matrix,  
                        train_targets, 
                        dev_feature_matrix,
                        dev_targets,
                        lr=1,
                        C=0.0,
                        batch_size=100,
                        max_steps=1000,
                        eval_steps=50):
    '''
    feel free to significantly modify the body of this function as per your needs.
    ** However **, you ought to make use of compute_gradients and update_weights function defined above
    return your best possible estimate of LR weights

    a sample code is as follows -- 
    '''

        #get the closed form solution as starting point
    c_r = C
    df_weights = analytical_solution(train_features, train_targets, c_r)
    print(df_weights)
    weights = df_weights
    gradient_i = weights
    converged = False
    step_size = lr
    tolerance = 1200
    features = train_feature_matrix 
    targets = train_targets
    val_features = dev_feature_matrix
    val_targets = dev_targets
    while not converged:
        counter=0
        error = 0
        error_eval = 0
        toggle_old = True
        error_old = 0
        toggle = True
        dev_loss = loss_fn(val_features, weights, val_targets,c_r)
        train_loss = loss_fn(features, weights, targets,c_r)
        print("dev loss: {} \t train loss: {}".format(dev_loss,train_loss))
        error += (dev_loss + train_loss)/2
        error_old = error
        error_eval = error_old
        for step in range(1,max_steps+1):

            features,targets = sample_random_batch(train_features,train_targets,batch_size)
            val_features,val_targets = sample_random_batch(dev_feature_matrix,dev_targets,batch_size)
            grad = compute_gradients(features, weights, targets)  
            weights = update_weights(weights, grad, step_size)
            

            if step%eval_steps == 0:
                dev_loss = loss_fn(val_features, weights, val_targets,c_r)
                train_loss = loss_fn(features, weights, targets,c_r)
                print("dev loss: {} \t train loss: {}".format(dev_loss,train_loss))
                error_eval = (dev_loss + train_loss)/2
                if error_eval < tolerance:
                    converged = True
                    break
                error += error_eval
                counter = counter+1

                

                #5 fold cross validation
                if counter == 5:
                    if toggle == True:
                        c_r = c_r + 0.000001
                    if toggle == False:
                        c_r = c_r - 0.000001
                    error = error/5
                    error_eval = error
                    error = 0
                    toggle_old = toggle
                    if error < error_old:
                        toggle = toggle_old
                    
                    if error > error_old:
                        toggle = !toggle
                        
                    toggle_old = toggle
                    error_old = error


                    #print("dev loss: {} \t train loss: {}".format(dev_loss,train_loss))


    return(weights)

In [33]:
    gradient_descent_soln = do_gradient_descent(train_features, 
                        train_targets, 
                        dev_features,
                        dev_targets,
                        1e-3,
                        0.000001,
                        32,
                        20000,
                        5)

14
[ -47.84794166    5.14092199   -5.40487476 1887.08507373  244.03920138
   11.02879444   14.16281222   -1.92149529    2.63845957 -127.94396024
  -18.81447284   -7.71459248   25.09225439   91.78053666]
dev loss: 35092.16935909341 	 train loss: 22502.20268550447
dev loss: 3617.618129470813 	 train loss: 1501.2233181599888
dev loss: 4110.245859000859 	 train loss: 3018.6955398123455
dev loss: 85931.78825639315 	 train loss: 22663.298990054343
dev loss: 1293.7625749640097 	 train loss: 27132.227183672163
dev loss: 1957.9345789548106 	 train loss: 1566.6904127018083
dev loss: 3903.470226041941 	 train loss: 1820.0470630980867
dev loss: 905.3199919060938 	 train loss: 1181.8355355221665


In [34]:
gradient_descent_soln

array([ -48.06083736,    4.9842618 ,   -5.47899193, 1886.85184935,
        243.99344925,   10.93668191,   14.12085947,   -1.99468213,
          2.45252461, -128.11269977,  -18.81982162,   -7.80033613,
         25.05583894,   91.60064416])

In [37]:
test_features = get_features("test.csv",False)
frp = (get_predictions(test_features, gradient_descent_soln))
frp

array([ 99.90678304, 440.80637908, -14.35620193, ...,  -3.33183013,
       -19.79507546,  82.93103167])

In [38]:
# display the array
print(frp)
  
# convert array into dataframe
DF = pd.DataFrame(frp)
  
# save the dataframe as a csv file
DF.to_csv("frp_prediction.csv")

[ 99.90678304 440.80637908 -14.35620193 ...  -3.33183013 -19.79507546
  82.93103167]
